# ChemBioSys and AquaDiva | Genome-resolved metagenomics workshop
## Functional genome mining - finding biosynthetic gene clusters associated with specialized metabolites

In this section of the workshop, we will provide an overview of biosynthetic gene clusters, the functions they code for in genomes, and explore some tools used to find them.

## So what are specialized metabolites and biosynthetic gene clusters?

Specialized metabolites, also often referred to as secondary metabolites (although not all secondary metabolites are so specialized) include antibacterial, antifungal, anticancer compounds, often produced by bacteria and fungi. These compounds represent many different compound classes, of which non-ribosomal peptide synthases (NRPS), polyketide synthases (PKS), ribosomally synthesised and post-translationally modified peptides (RiPP), terpenes, bacteriocins are most well known. These compounds have historically been discovered through lab-based bioassays combined with liquid chromatography, mass spectometry and nuclear magnetic resonance (NMR) spectroscopy. 

Along with high throughput sequencing becoming a cost-effective and widely used method in molecular biology, new avenues for discovery of specialized metabolites, and also studying the evolution and distribution of genetic content associated with these metabolites have been developed. 



## Exercise - Explore a biosynthetic gene cluster.

[Open the Mibig database here](https://mibig.secondarymetabolites.org/repository)

Search the repository for an antibiotic - perhaps one that you have taken in the past, or one that interests you.

Look through the information provided, like the length of the BGC and the number and type of genes. If you scroll down you will find some more information, such as the organism or genome where it was recorded. 

The menu will take you through the compound structure, NRPS or PKS modules and accession numbers for individual genes.

How many genes are present in the BGC you selected?
How long is the cluster?
Do you find some regulatory genes, transporter genes and genes of unknown function, along with the core biosynthesis genes?
What organisms produce it?
Is it an NRPS or PKS or something else?


## Using antiSMASH to detect biosynthetic gene clusters in metagenome assembled genomes

We will use antiSMASH (antibiotic and Secondary Metabolite Analysis Server), which is currently the premier tool for detection of biosynthetic gene clusters.

antiSMASH will search all genes in your genome against a set of core BGC-related genes in its database. [open this link for more infomation.](https://docs.antismash.secondarymetabolites.org/understanding_output/regions/) Upon detection any of these genes, it will then use a set of "rules" for each type of BGC it detects. These rules are essentially a set of HMM for each rule and upon detection, the cluster gets identified according to the rule. 

For example, in PKS gene clusters contain regions called ketosynthase (KS) and acyltransferase (AT) domains which are conserved to some degree. For this reason it is possible to build HMMs to detect them. The same is true for NRPS clusters, which have condensation (CDS) and A- (adenylation) domains which enable its detection.

You can see all of the antiSMASH rules [here](https://docs.antismash.secondarymetabolites.org/glossary/#clustertypes). If you are interested in seeing more of the inner workings, open the rule files in the [antiSMASH source.](https://github.com/antismash/antismash/blob/master/antismash/detection/hmm_detection/cluster_rules/strict.txt)
 
[Now open the antiSMASH webserver.](https://antismash.secondarymetabolites.org/#!/start)

[And lets look at a typical set of results.](https://antismash.secondarymetabolites.org/upload/bacteria-75a3865d-f2be-4952-bf50-07897c40434f/index.html#r1c1)

## antiSMASH input options

### KnownClusterBlast

The identified clusters are searched against the MIBiG repository. MIBiG is a hand curated data collection of biosynthetic gene clusters, which have been experimentally characterized.### 

ClusterBlast

The identified clusters are searched against a comprehensive gene cluster database and similar clusters are identified. The algorithm used here is inspired by MultiGeneBlast. It runs BlastP using each amino acid sequence from a detected gene cluster as a query on a large database of predicted protein sequences from secondary metabolite biosynthetic gene clusters, and pools the results to identify the gene clusters that are most homologous to the gene cluster that was detected in your query nucleotide sequence. Please note that selecting this option increases the runtime significan### tly.

SubClusterBlast

The identified clusters are searched against a database containing operons involved in the biosynthesis of common secondary metabolite building blocks (e.g. the biosynthesis of non-proteinogenic amino###  acids).

ActiveSiteFinder

Active sites of several highly conserved biosynthetic enzymes are detected and variations of the active sites a### re reported.

Cluster Pfam analysis

Each gene product encoded in the detected BGCs is analyzed against the PFAM database. Hits are annotated in the final Genbank/EMBL files that can be downloaded after the analysis is finished. Please note that these results are not displayed on the antiSMASH HTML results page but they are present in the results genbank file that can be downloaded. Also, selecting this option normally increa### ses the runtime.

Pfam-based GO term annotation

This is annotating the Cluster Pfam analysis described above with GO term annotations. Please note that these results are not displayed on the antiSMASH HTML results page but they are present in the results genbank file that can be downloaded (see "Understanding the output" section of the documentation for instructions on how to download the results). Also, selecting this option normally increases the runtime

## Running antiSMASH standalone

Submitting your metagenome bins individually to the antiSMASH webserver will be incredibly time-consuming and counterproductive. Instead, we use the antiSMASH standalone version. This also provides the opportunity for further customizing your analysis.

In [ ]:
cd ~/data/workshop/09_ANTISMASH
# activate the antismash conda environment
conda activate antismash
#
# View the antiSMASH input options
antismash -h

## Running antiSMASH - important input parameters

As you can you, there are many input options. Most of these depend on the purpose of your analysis, like if you are interested in possibly identifying a BGC using known references (less likely in metagenomics), counting the number of biosynthetic gene clusters and getting to most hits possible, or doing further functional exploration and looking for subclusters and other interesting features. 

If your are not providing an already annotated genbank (.gbk) file, you also need to specify a genefinding tool using the --genefinding-tool option. In our case, we will use "--genefinding-tool prodigal-m", which refers to well-known gene prediction tool "prodigal" in metagenomic mode and because our input is unannotated fasta files from your binning steps. 

You could also provide your own gene calling information by providing the gff files along with your input fasta files.

Other considerations:

- Inluding whole genome HMMer
- using a --minlength cutoff
- cluster comparison options (results to be interpreted with caution)

Thus, the command to run antiSMASH looks like this:

### antismash input_file --output-dir antismash_result/bin_name --clusterhmmer --cc-mibig --cb-general --cb-knownclusters --pfam2go --rre --minlenth 7500 --genefinding-tool prodigal-m

However, since we have many genomes, we will run antiSMASH using a for-loop. Essentially, this means that we execute antiSMASH on each of our fasta files (from your genome bins).

You might see the warning about long sequence headers - you can ignore this, but keep in mind your contig names are now different. Version 7 of antiSMASH allows disabling this function.

In [ ]:
# make a directory from where your antismash output will be stored
mkdir antismash_result
for fasta_file in ./input/*.fa; do base=$(basename "$fasta_file" .fa); antismash "$fasta_file" --output-dir antismash_result/$base --clusterhmmer --cc-mibig --minlength 7500 --cb-general --cb-knownclusters --pfam2go --rre --genefinding-tool prodigal-m ; done

### Lets explore the antiSMASH output

In [ ]:
cd antismash_result
ls
cd Bin_3
ls

The way we specified that antiSMASH uses an output directory with the same name as our genome bins means that we now have a directory with the results for each.

In the output, you will see the following:

Bin_3.gbk                         - this file contains ORFs identified and annotated according to the information available 
Bin_3.json                        - if you are good at Python you will want to explore the treasure trove of information stored in antiSMASH json files.
NZ_CP009530.1.region001.gbk       - these are the files that contain most details about the clusters, and are most important for most of the downstream processing.
NZ_CP009530.1.region002.gbk
NZ_CP009530.1.region003.gbk
index.html                        - opening this file will present results exactly as we saw it earlier on the antiSMASH webserver. This could be very useful if you have few genomes and target specific BGCs
knownclusterblast.txt
clusterblast.txt

Next step would be to tabulate the antiSMASH output to really see what we got. To do this we will use two Python scripts borrowed from Zacchary Reitz (https://github.com/zreitz/multismash). One will create a table with BGC counts per genome, and the other will provide more information on specific BGCs

In [ ]:
head -n 30 NZ_CP009530.1.region001.gbk

In [ ]:
cd ../..
python count_regions.py example_output/ BGC_counts.tsv
python tabulate_regions.py example_output/ BGCs_info.tsv

In [ ]:
head -n 60 BGC_counts.tsv

In [ ]:
head -n 60 BGC_tab.tsv

You can open the two files generated above and see what your output looks like?

How many clusters were detected?
Which type of BGCs were most common?
Which genome had the most BGCs?

In [ ]:
wget https://raw.githubusercontent.com/Zander22/metallophore/main/plot_BGCs.R
conda deactivate
conda activate r
Rscript ./plot_BGCs.R

Well done, you have now completed your first dive into antiSMASH, explored its capabilities and its output.

### Other BGC mining tools

If your main goal is mining metagenomes or metagenome bins to identify as many BGCs as possible, you might be interested to combine different approaches. 

### Some other BGC identification tools include:
GECCO: https://gecco.embl.de/ - An alternative, de novo approach to find BGCs[DeepBG: (https://github.com/Merck/deepbg - 
Deep learning approach for BGC detection[
biosyntheticSdPA: ](https://cab.spbu.ru/software/biosyntheticspade - s reconstructs BGCs in assembly graphs from genomic and metagenomics data set

# #
Class-specific BGC detet[
RiPP RODEO: https://ripp.rodeo/index.html - RiPP discovery and genomic context
BAGEL:4https://github.com/annejong/BAGEL4 -  search DNA for bacteriocins and RiPPs
DeepRi:phttp://deepripp.magarveylab.ca/ - p neural network-based detection of RiPPs
decRiPP:thttps://github.com/Alexamk/decRiPPter - er designed to detect novel RiPP BGCs not restricted to specific genetic markers
trans:Phttps://github.com/chevrm/transPACT - ACT trans-acetyltransferase polyketide synthase detectionsthase d

If you are interested in any of these tools, perhaps you can try them out in th BYOD session.etection

## Clustering BGCs

There are many tools available which allows the clustering of BGCs. There are many reasons to do this, such as aiding in identification, doing evolutionary mining analysis or assessing whether BGCs are potentially divergent from anything knwon.

Tools you could use include: 
[BiG-SCAPE](https://bigscape-corason.secondarymetabolites.org/) 
[CLUST-O-MATIC](https://github.com/Helmholtz-HIPS/clustomatic_source)
[BiG-SLiCE](https://github.com/medema-group/bigslice)

### BiG-SLiCE: Biosynthetic Gene clusters - Super Linear Clustering Engine
[Paper here](https://academic.oup.com/gigascience/article/10/1/giaa154/6092777)

BiG-SLiCE allows clustering thousands of BGCs very quickly and build databases based on the results. It also allows querying your own set of BGCs against a dataset contain 1.2 million BGCs.

Here we will use the BiG-FAM database as a refernce to compare the potential novelty of BGCs in our dataset.

The reference database containing 1.2 million BGCs is also available online and named (BiG-FAM.)[https://bigfam.bioinformatics.nl/home) Let's open and explore that.

Here is a description from the website: 

    The Biosynthetic Gene Cluster Family (GCF) database is an online repository for "homologous" groups of biosynthetic gene clusters (BGCs) putatively encoding the production of similar specialized metabolites.     By taking large-scale, global collections of BGCs identified from currently available genomes and MAGs as a data source, BiG-FAM provides an explorable "atlas" of microbial secondary metabolic diversity to b     browse and search biosynthetic diversity across taxa. BiG-FAM facilitates querying putative BGCs to rapidly find their position on the diversity map and gain a better understanding of their novelty or            functions, based on relationships with other known and predicted BGCs from publicly available data.
    For the Big-SLiCE analysis, we will create a new directory with all the BGC .gbks from our earlier antiSMASH output. The reference database is already prepared, and named "full_run_result"

Here it is also important to consider clustering thresholds (how similar gene clusters are to be clustered together). Furthermore, when doing the BiG-SLiCE query analysis the (euclidean) distance between our BGCs and those in the database is important. For the moment, we use the thresholds defined in the paper as distance.

### Make sure to speak to us later if you are interested in setting up BiG-SLiCE to query BGCs on our own server or laptop.

### Here are the steps for this analysis:

In [ ]:
conda activate bigslice
cd ../10_BIGSLICE
bigslice --query example_input/ --run_id 6 --query_name workshop_12092023 full_run_result/

### BiG-SLiCE results

The results are output to a very large database, containing 1.2 million BGCs with their taxonomy and already clustered into gene cluster families (GCFs). These data are stored in a sqlite database, which can house very large amounts of data, but it is not very intuitive to use for new users.

On your own server or computer, you can explore the results using the Flask webserver as follows. Instructions can be found [here](https://github.com/medema-group/bigslice/tree/master/misc/input_folder_template)

The BiG-SLiCE output provides distance values - distance between your BGCs and the best-matching gene-cluster family in the database. A distance of < 900 is considered "related" to the closest reference, a distance > 900 and < 1800 means it is distantly related to the GCF in the reference, and a distance of > 1800 means extremely divergent from any known references. 

I will provide a python script to extract the necessary information from the BiG-SLiCE analysis, which we will use to make a figure in R.

In [ ]:
wget https://raw.githubusercontent.com/Zander22/metallophore/main/plot_BGC_distances.R
wget wget https://raw.githubusercontent.com/Zander22/metallophore/main/extract_distances.py
#
cd full_run_result
#
python get_report.py
python extract_distances.py
#
cp exported_full_tables_Sp_Report/merged_table.txt ../
#
cd ..
#
conda deactivate
conda activate r
#
Rscript ./plot_BGC_distances.R

How divergent were the BGCs in this dataset? Do you think there are any that are completely novel?

If time allows, we will now use a different method to cluster BGCs as a means to dereplicate, identify and visualize the presence of difference BGCs across genomes.

We will use the [CLUST-O-MATIC](https://github.com/Helmholtz-HIPS/clustomatic_source) tool from the Ziemert lab. 

The db2fasta.py script will convert our cluster (region) .gbk files to fasta files and then we run clust-o-matic.
db2fasta takes directories labelled according to genomes, with the cluster .gbk files inside, and will create a single fasta file.

Then running clust-o-matic, the input will be the fasta file created, and the clustering threshold.

In [ ]:
conda deactivate
conda activate clustomatic
cd ../11_CLUSTOMATIC
# first we cluster the BGCs from the genome bins and see whether any BGCs are shared between genome bins
python gb2fasta.py input_clusters/ BGCs_genomes.faa
python clustomatic.py BGCs_genomes.faa 0.25 > BGCs_genomes_clusters.txt

In [ ]:
head -n 50 BGCs_genomes_clusters.txt

You can see in the output the genome name in the first column, and the gene cluster family number in the second column. We will parse these in R and plot a heatmap

In [ ]:
# now we cluster our genome bins together with the MiBiG database to see whether this approach allows us to identify some of these clusters
python gb2fasta.py input_clusters/ BGCs_mibig.faa
python clustomatic.py BGCs_mibig.faa 0.25 > mibig_clusters.txt

In [ ]:
head -n 50 mibig_clusters.txt

Again as above, but this time MiBiG BGCs are added to the output. We will filter these to any possible hits to your genome bins and make another heatmap in R.

In [ ]:
wget https://raw.githubusercontent.com/Zander22/metallophore/main/heatmap_BGC_genomes.R
wget https://raw.githubusercontent.com/Zander22/metallophore/main/heatmap_BGC_mibig.R
conda deactivate
conda activate r
Rscript ./heatmap_BGC_genomes.R
Rscript ./heatmap_BGC_mibig.R

## A few tools to further explore the function, evolution or relevance of your BGCs

ARTS: http://arts.ziemertlab.com/ - Antibiotic Resistant Target Seeker, a tool to help identify resistance genes within BGCs.
NaPDoS: https://npdomainseeker.sdsc.edu/napdos2/napdos_home_v2.html -  classification of ketosynthase (KS) and condensation (C) domains
EvoMinng: (https://github.com/nselem/EvoMining/wik -g genome-mining and visualization approach for biosynthetic enzyme discovery that incorporates evolutionary principl[s
AutLST: ](https://automlst.ziemertlab.coAutoM/)LS- T automatic generation of a species phylogeny with reference organi

ARTS is a really cool tool, and could potentially aid you in identifying whether some of your novel BGCs code for antimicrobial compounds, based on the assumption that BGCs will have antibiotic resistance genes location on or near them. If there is time. Open the link...sms

## Summary

You have:
 - mined your genome bins for BGCs and explored its output
 - clustered your BGCs to a database with 1.2 million BGCs and determined whether any of them are novel clusters
 - clustered your BGCs to try and identify them, and dereplicate them in your dataset.